In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [3]:
import pandas as pd
import numpy as np

Import our white and red wine data from the csvs

# Read the CSV and Perform Basic Data Cleaning

In [4]:
red_wine_df = pd.read_csv("Resources/winequality-red.csv", sep=';')
white_wine_df = pd.read_csv("Resources/winequality-white.csv", sep=';')
white_wine_df.info()
red_wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
fixed acidity           1599 non-null float64
volatile acidity        1599 non-null float64
citric acid             1599 non-null float64
residual sugar          1599 non-null float64

## Balanced (Normalized) Data for White Wine Data (1599 data points) and Red Wine Data (1599 data points) 

In [5]:
#take a random sample of white wines to match the red wine data
random_white_df = white_wine_df.sample(1599,random_state=42).reset_index(drop=True)

Only balanced values are baseline condition we can use:

In [6]:
#if we want our only features of the model to have balanced white wine and red wine data, we use this data frame:
baseline_red_df = red_wine_df.copy()
baseline_white_df = random_white_df.copy()
only_balanced_df = pd.concat([baseline_red_df, baseline_white_df])
only_balanced_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.7,0.34,0.31,16.4,0.051,20.0,146.0,0.99834,3.06,0.54,9.1,5
1595,9.3,0.31,0.49,1.3,0.042,34.0,147.0,0.99480,3.11,0.46,9.8,5
1596,6.4,0.17,0.27,6.7,0.036,88.0,223.0,0.99480,3.28,0.35,10.2,6
1597,7.5,0.29,0.36,15.7,0.050,29.0,124.0,0.99680,3.06,0.54,10.4,5


In [ ]:
#To check the counts for each quality score in `only_balanced_df`
#only_balanced_df.groupby('quality').count()

In [ ]:
#Suppose we want to take specific ranges of quality scores in only_balanced_df (which does not include colors)

range_balanced_df = only_balanced_df.loc[\
(only_balanced_df["quality"] <= 8 ) & \
(only_balanced_df["quality"] >= 3 )  \
] 

#range_balanced_df

In [ ]:
#To check the counts for each quality score in `range_balanced_df`
#range_balanced_df.groupby('quality').count()

Added Color with get dummies (Or in case we want to remove it)

In [ ]:
#if we want to include wine colors as features in our model:
color_red_df = red_wine_df.copy()
color_white_df = random_white_df.copy()
color_red_df["color"] = "red"
color_white_df["color"] = "white"
colors_df = pd.concat([color_red_df, color_white_df])
#if we only want to use the balanced data and include the colors of wine:
#0 is red, 1 is white
dummy_colors_df = pd.get_dummies(colors_df, columns=['color'])
dummy_drop_colors_df = dummy_colors_df.drop(columns=["color_red"]).copy()
dummy_rename_colors_df = dummy_drop_colors_df.rename(columns={"color_white":"colors"})
#dummy_rename_colors_df

If we want to hypertune further, and choose a specfic range of quality scores, so our model predicts a smaller range of quality scores

In [ ]:
#For our model that includes our `colors` feature, if we want include a range of quality scores:
range_colors_df = dummy_rename_colors_df.loc[\
(dummy_rename_colors_df["quality"] <= 6 ) & \
(dummy_rename_colors_df["quality"] >= 3 )  \
] 
#range_colors_df

In [ ]:
#To show which range we are using
range_colors_df.groupby('quality').count()

Add buckets to given range of quality values

## Created another dataframe that moves quality scores into bins

Created bins for balanced data called: "terrible (1), mediocre (2), great (3) and terrific (4)"
The quality scores range from 0 to 4, 5 to 6, and 7 to 10. We will call them 1,2,3,4

In [ ]:
bin_balanced_df = only_balanced_df.copy()
bins = [0, 4, 5, 6, 8]
group_names = [1, 2, 3, 4]
bin_balanced_df["bin_quality"] = pd.cut(bin_balanced_df["quality"], bins, labels=group_names)
bin_qual_df = bin_balanced_df.drop(columns="quality")
bin_qual_df

In [ ]:
bin_qual_df.groupby("bin_quality").count()

Create another dataframe that adds colors with bins to test our model.

In [ ]:
bins_colors_df = dummy_rename_colors_df.copy()
bins = [0, 4, 5, 6, 8]
group_names = [1, 2, 3, 4]
bins_colors_df["bin_quality"] = pd.cut(bins_colors_df["quality"], bins, labels=group_names)
new_bins_colors_df = bins_colors_df.drop(columns="quality")
new_bins_colors_df

In [ ]:
new_bins_colors_df.groupby("bin_quality").count()

In [ ]:
new_col_bal_df = only_balanced_df.copy()
new_col_bal_df["fsd/tsd"] = new_col_bal_df["free sulfur dioxide"]/new_col_bal_df["total sulfur dioxide"]
new_col_bal_df["alc/den"] = new_col_bal_df["alcohol"]/new_col_bal_df["density"]
new_col_bal_df["sug/den"] = new_col_bal_df["residual sugar"]/new_col_bal_df["density"]
col_bal_df = new_col_bal_df.drop(columns=['free sulfur dioxide','total sulfur dioxide',"alcohol","density","residual sugar"])
# col_bal_df

feat_bin_balanced_df = col_bal_df.copy()
bins = [0, 4, 5, 6, 8]
group_names = [1, 2, 3, 4]
feat_bin_balanced_df["bin_quality"] = pd.cut(feat_bin_balanced_df["quality"], bins, labels=group_names)
feat_bin_qual_df = feat_bin_balanced_df.drop(columns="quality")
new_feat_df = feat_bin_qual_df.drop(columns=["pH","citric acid"])
new_feat_df

# Select your features (columns)

In [7]:
# Set features. Drop quality, y value. 
X = only_balanced_df.drop(columns=['quality'])
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


# Create a Train Test Split

Use `quality` for the y values

In [8]:
from sklearn.model_selection import train_test_split
#80% train, 20% test. y is this one column
y = only_balanced_df['quality']
#random state 42 will have same picks for x test and y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1435,6.8,0.26,0.40,7.5,0.046,45.0,179.0,0.99583,3.20,0.49,9.3
977,5.9,0.26,0.21,12.5,0.034,36.0,152.0,0.99720,3.28,0.43,9.5
533,10.3,0.27,0.24,2.1,0.072,15.0,33.0,0.99560,3.22,0.66,12.8
1061,9.1,0.40,0.50,1.8,0.071,7.0,16.0,0.99462,3.21,0.69,12.5
1027,7.4,0.18,0.36,13.1,0.056,72.0,163.0,1.00000,3.42,0.35,9.1


# Train the Model using Random Forest 



In [10]:
#Train the model
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
#Test the model
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        19
           5       0.67      0.72      0.69       243
           6       0.59      0.65      0.62       270
           7       0.63      0.55      0.59        92
           8       1.00      0.07      0.12        15

    accuracy                           0.63       640
   macro avg       0.48      0.33      0.34       640
weighted avg       0.62      0.63      0.61       640



/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:

feature_list = list(X.columns)
feature_imp = pd.Series(rfc.feature_importances_,index=feature_list).sort_values(ascending=False)
print("The Accuracy Score =", accuracy_score(y_test, predictions))
feature_imp

The Accuracy Score = 0.628125


alcohol                 0.136286
volatile acidity        0.104488
density                 0.100351
sulphates               0.092727
total sulfur dioxide    0.090937
chlorides               0.086812
residual sugar          0.080950
pH                      0.079175
free sulfur dioxide     0.077730
citric acid             0.077498
fixed acidity           0.073046
dtype: float64

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'feat_eng_new_cols_df.sav'
joblib.dump(predictions, filename)